# Creating Commitment Transactions with HTLC

In this section, we'll build commitment transactions with an in-flight HTLC from scratch using Python. We'll break down each component of the transaction, explain how it's constructed and signed, and go over the messages exchanged between peers to coordinate the required information. Finally, we'll test everything locally using Bitcoin Core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'
- Finite Fields, Elliptic Curves and Serialization e.g. [Programming Bitcoin](https://github.com/jimmysong/programmingbitcoin), in particular [Chapter 1](https://github.com/jimmysong/programmingbitcoin/blob/master/ch01.asciidoc), [Chapter2](https://github.com/jimmysong/programmingbitcoin/blob/master/ch02.asciidoc), [Chapter 3](https://github.com/jimmysong/programmingbitcoin/blob/master/ch03.asciidoc) and [Chapter 4](https://github.com/jimmysong/programmingbitcoin/blob/master/ch04.asciidoc).

## HTLC (Hashed Time Locked Contracts)

Once the channel is established, it can be used to make payments via HTLCs (Hashed Time-Locked Contracts).

Updates to the channel state are sent in batches: one or more `update_ messages` are exchanged before a `commitment_signed` message is sent. In the diagram below, we show only one`update_message` per `commitment_signed` for simplicity:

    +-----------+                            +-----------+
    |           |------ update_add_htlc ---->|           |
    |           |----- commitment_signed --->|           |
    |   Alice   |                            |    Bob    |
    |           |<----- revoke_and_ack ------|           |
    |           |<---- commitment_signed ----|           |
    |           |------ revoke_and_ack ----->|           |
    |           |                            |           |
    +-----------+                            +-----------+

The messages are defined in the [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#adding-an-htlc-update_add_htlc).

### The `update_add_htlc` Message

Alice sends the `update_add_htlc`  message to Bob, which includes the details of the HTLC being offered:
- channel_id
- id (htlc counter starting in zero)
- amount_msat
- payment_hash
- cltv_expiry
- onion_routing_packet

### The `commitment_signed` Message

Shortly after sending the `update_add_htlc` message, Alice commits to the new channel state so that the HTLC can be safely included by Bob. At this point, Bob has the HTLC information and has constructed a new commitment transaction, but he hasn't yet received Alice's signature for it.

Alice sends a `commitment_signed`message to Bob, which includes her signature for the new commitment transaction as well as the HTLC output it contains:
- channel_id
- signature
- num_htlcs
- htlc_signature

### The `revoke_and_ack` Message

Now that Bob has a new signed commitment, he needs to acknowledge it and revoke the previous one. He does this by sending a`revoke_and_ack` message to Alice:

- channel_id
- per_commitment_secret
- next_per_commitment_point

Finally, Bob sends a `commitment_signed` message to Alice, and Alice responds with a `revoke_and_ack` message. At this point, both parties have fully signed the new commitment transaction.

## Setup
### Requirements

For this exercise we'll run the previous notebook to create the channel funding transaction beetween Alice and Bob.

**You'll need to edit these next line for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 2 - Commitment Transactions/First Commitment Transactions.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

## Transaction Inputs

Let’s assume Alice has previously sent 1 million satoshis to Bob via an HTLC, which has already been settled. Now, Alice is sending 0.5 million satoshis to Bob.

We can now begin constructing the third set of commitment transactions for both Alice and Bob. Since these transactions spend the same funding output, their inputs are nearly identical. The only distinction lies in the **obscured commitment number**.

To allow both parties to track the channel state, this commitment number is encoded into the `locktime` and `sequence` fields of the commitment transaction. However, rather than using the plain commitment number, it is obfuscated by XORing it with a shared secret derived deterministically from the payment basepoints of both parties.

In [2]:
# Third Commitment number
commitment_number = 3 

# Obscured commitment number is result of xor operation
# See the First Commiment Transactions to know how to_obscure_int_lower48 was generated 
commitment_number_obscured = to_obscure_int_lower48 ^ commitment_number
print("Commitment Number Obscured: " + hex(commitment_number_obscured))

# sequence: upper 8 bits are 0x80, lower 24 bits are upper 24 bits of the obscured commitment number
# Extract the upper 24 bits of the obscured commitment number
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (1 byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian
sequence = sequence[::-1]
inputs = (
    channel_funding_txid
    + channel_funding_txindex
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)
print("Inputs: " + inputs.hex())

Commitment Number Obscured: 0xb433fd43a66c
Inputs: 58daedb84cb5a96c3d7899fed40160e9fc980168b82c42670fdb9de6820b18ca0100000000fd33b480


## Transaction Outputs

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the outputs as following:

* For every offered HTLC, if it is not trimmed, add an offered HTLC output.
* For every received HTLC, if it is not trimmed, add an received HTLC output.
* If the to_local amount is greater or equal to dust_limit_satoshis, add a to_local output.
* If the to_remote amount is greater or equal to dust_limit_satoshis, add a to_remote output.
* If option_anchors applies to the commitment transaction:
    * if to_local exists or there are untrimmed HTLCs, add a to_local_anchor output
    * if to_remote exists or there are untrimmed HTLCs, add a to_remote_anchor output

Since this is Alice’s first commitment transaction, it will have no outputs to Bob. That’s because Alice is the funder of the channel and isn't sending any sats to Bob initially. If she wanted to transfer funds at channel opening, she could have set the `push_msat` field in the `open_channel` message, specifying the amount in millisatoshis.

Additionally, because the channel hasn’t been opened yet, there are no offered or received HTLCs—so no HTLC outputs will be created either.

As a result, the outputs in the first commitment transaction will be as follows:

Alice first Commitment Transaction will have two outputs:
* to_local_anchor_output
* to_local_output

Bob first Commitment Transaction will have two outputs:
* to_remote_anchor_output
* to_remote_output

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) specifies that transaction outputs must be sorted by value, from smallest to largest. Because of this, the to_local_anchor_output appears first—it has a fixed amount of 330 sats, which corresponds to the default dust limit for P2WSH outputs.

**Anchor outputs** exists to prevent a malicious peer from attaching low-fee child transactions, which could block the victim from getting the commitment transaction confirmed in time. This defense was made possible by a change introduced in Bitcoin Core 0.19: [bitcoin/bitcoin#15681](https://github.com/bitcoin/bitcoin/pull/15681). It’s also the reason why all non-anchor outputs in the commitment transaction are CSV (CheckSequenceVerify)-locked: to delay their spendability and allow time for fee bumping via the anchor. The anchor outputs feature is optional and can be enabled only if both channel peers support it. However, starting with [Eclair v0.11.0](https://github.com/ACINQ/eclair/blob/master/docs/release-notes/eclair-v0.11.0.md) stop accepting channels that don't support anchor outputs.

A **trimmed HTLC** is an HTLC that does not get included in the commitment transaction because its value is too low to be economically spent. Specifically, if the value of the HTLC is below the dust limit plus the estimated fee required to claim it, the output is trimmed—in other words, left out of the transaction entirely.

This mechanism helps avoid bloating the commitment transaction with outputs that would either be unspendable or cost more in fees than they're worth.

## The Per Commitment Point
As we saw on Chapter 0, the I'th per commitment secret must match the output of this algorithm:
```
generate_from_seed(seed, I):
    P = seed
    for B in 47 down to 0:
        if B set in I:
            flip(B) in P
            P = SHA256(P)
    return P
```
Where "flip(B)" alternates the (B mod 8) bit of the (B div 8) byte of the value. The first secret used must be index 281474976710655 (0xFFFFFFFFFFFF), and from there, the index is decremented.

The per_commitment_point is generated using elliptic-curve multiplication:

```
per_commitment_point = per_commitment_secret * G
```

Alice and Bob create their per-commitment point and exchange it using the `revoke_and_ack` messages.

In [3]:
# The first secret used must be index 281474976710655 (0xFFFFFFFFFFFF), the third one is 281474976710653
alice_per_commitment_secret = generate_from_seed(alice_per_commitment_seed, 281474976710653)
alice_per_commitment_point_uncompressed = int.from_bytes(alice_per_commitment_secret, byteorder="big") * G
alice_per_commitment_point = compress_pubkey(alice_per_commitment_point_uncompressed)

print(f"Alice Per Commitment Secret: {alice_per_commitment_secret.hex()}")
print(f"Alice Per Commitment Point: {alice_per_commitment_point}")

# The first secret used must be index 281474976710655 (0xFFFFFFFFFFFF), the third one is 281474976710653
bob_per_commitment_secret = generate_from_seed(bob_per_commitment_seed, 281474976710653)
bob_per_commitment_point_uncompressed = int.from_bytes(bob_per_commitment_secret, byteorder="big") * G
bob_per_commitment_point = compress_pubkey(bob_per_commitment_point_uncompressed)

print(f"Bob Per Commitment Secret: {bob_per_commitment_secret.hex()}")
print(f"Bob Per Commitment Point: {bob_per_commitment_point}")


Alice Per Commitment Secret: 7d474813d764ca1273ad85e3f36ca1926401a8f76e576533f0232801c96af14d
Alice Per Commitment Point: 028a29fc294e5cc904c607cbe2d66bc3e3f0d5131ba3f9085eb01b25df0a5bdd4a
Bob Per Commitment Secret: 3dec4e412d557115ad23549b60f2bb4fc8a1215bead2c8ade3ed2cf1d10450da
Bob Per Commitment Point: 0267c6f101ca3ffa7d9e2aba0ca51e63af33a7b88b19e80a04a2deb9807447e963


## Key Derivations

Each commitment transaction uses unique keys: `localpubkey`, `local_htlcpubkey`, `remote_htlcpubkey`, `local_delayedpubkey`, and `remote_delayedpubkey`. These public keys are derived by simply adding a per-commitment point to their respective base points. As defined at Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G

```

- The `localpubkey` uses the local node's `payment_basepoint`;
- The `local_htlcpubkey` uses the local node's `htlc_basepoint`;
- The `remote_htlcpubkey` uses the remote node's `htlc_basepoint`;
- The `local_delayedpubkey` uses the local node's `delayed_payment_basepoint`;
- The `remote_delayedpubkey` uses the remote node's `delayed_payment_basepoint`.

The `revocationpubkey` is a blinded public key. When the local node prepares a new commitment transaction for the remote node, it derives the `revocationpubkey` by combining its own `revocation_basepoint` with the remote node’s `per_commitment_point`.

```
revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
```

Later, when the remote node revokes that commitment by revealing the corresponding `per_commitment_secret`, the local node can derive the `revocationprivkey`. At that point, it possesses both secrets needed for the derivation: the `revocation_basepoint_secret` and the `per_commitment_secret`.

This construction ensures that neither the node providing the `basepoint` nor the node providing the `per_commitment_point` can derive the private key on their own—each requires the other’s secret.

```
revocationprivkey = revocation_basepoint_secret * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_secret * SHA256(per_commitment_point || revocation_basepoint)
```

In [4]:
# ALICE DELAYED PUBKEY
# alice_delayed_pubkey = alice_delayed_payment_basepoint + SHA256(per_commitment_point || alice_delayed_payment_basepoint) * G
# alice_delayed_payment_basepoint is generate by alice and sent into the `open_channel` message
# per_commitment_point is generated by alice and sent into the `revoque_and_ack` message
# Compute SHA256(per_commitment_point || basepoint)
sha_output = sha256(bytes.fromhex(alice_per_commitment_point) + bytes.fromhex(alice_delayed_payment_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# do the math to get the alice_delayed_pubkey
alice_delayed_pubkey_point = decompress_point(alice_delayed_payment_basepoint_pubkey) + sha_int * G
alice_delayed_pubkey = compress_pubkey(alice_delayed_pubkey_point)
print(f"Alice Delayed Pubkey: {alice_delayed_pubkey }")

# BOB DELAYED PUBKEY
# bob_delayed_pubkey = bob_delayed_payment_basepoint + SHA256(bob_per_commitment_point || bob_delayed_payment_basepoint) * G
# bob_delayed_payment_basepoint is generate by bob and sent into the `accept_channel` message
# per_commitment_point is generated by bob and sent into the `revoke_and_ack` message
# Compute SHA256(per_commitment_point || basepoint)
sha_output = sha256(bytes.fromhex(bob_per_commitment_point) + bytes.fromhex(bob_delayed_payment_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# do the math to get the bob_delayed_pubkey
bob_delayed_pubkey_point = decompress_point(bob_delayed_payment_basepoint_pubkey) + sha_int * G
bob_delayed_pubkey = compress_pubkey(bob_delayed_pubkey_point)
print(f"Bob Delayed Pubkey: {bob_delayed_pubkey }")

# ALICE HTLC PUBKEY
# alice_htlc_pubkey = alice_htlc_basepoint + SHA256(alice_per_commitment_point || alice_htlc_basepoint) * G
# alice_htlc_basepoint is generate by alice and sent into the `open_channel` message
# per_commitment_point is generated by alice and sent into the `revoke_and_ack` message
# Compute SHA256(per_commitment_point || basepoint)
sha_output = sha256(bytes.fromhex(alice_per_commitment_point) + bytes.fromhex(alice_htlc_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# do the math to get the alice_htlc_pubkey
alice_htlc_pubkey_point = decompress_point(alice_htlc_basepoint_pubkey) + sha_int * G
alice_htlc_pubkey = compress_pubkey(alice_htlc_pubkey_point)
print(f"Alice HTLC Pubkey: {alice_htlc_pubkey }")

# BOB HTLC PUBKEY
# bob_htlc_pubkey = bob_htlc_basepoint + SHA256(bob_per_commitment_point || bob_htlc_basepoint) * G
# bob_htlc_basepoint is generate by bob and sent into the `accept_channel` message
# per_commitment_point is generated by bob and sent into the `revoke_and_ack` message
# Compute SHA256(per_commitment_point || basepoint)
sha_output = sha256(bytes.fromhex(bob_per_commitment_point) + bytes.fromhex(bob_htlc_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# do the math to get the alice_htlc_pubkey
bob_htlc_pubkey_point = decompress_point(bob_htlc_basepoint_pubkey) + sha_int * G
bob_htlc_pubkey = compress_pubkey(bob_htlc_pubkey_point)
print(f"Bob HTLC Pubkey: {bob_htlc_pubkey }")


# ALICE REVOCATION PUBKEY
# revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
# Derived from local basepoint + remote per-commitment point
# Compute SHA256(revocation_basepoint || per_commitment_point) REVOCATION PUBKEY
ha_output = sha256(bytes.fromhex(alice_revocation_basepoint_pubkey) + bytes.fromhex(bob_per_commitment_point))
sha_int = int.from_bytes(sha_output, 'big') % n
# now multiply it by revocation_basepoint
part1 = decompress_point(alice_revocation_basepoint_pubkey) * sha_int
# Compute SHA256(per_commitment_point || revocation_basepoint)
sha_output = sha256(bytes.fromhex(bob_per_commitment_point) + bytes.fromhex(alice_revocation_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# multiply it by per_commitment_point
part2 = decompress_point(bob_per_commitment_point) * sha_int
# sum the results
alice_revocation_pubkey_point = part1 + part2
alice_revocation_pubkey = compress_pubkey(alice_revocation_pubkey_point)
print(f"Alice Revocation Pubkey: {alice_revocation_pubkey }")

# BOB REVOCATION PUBKEY
# revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
# Derived from local basepoint + remote per-commitment point
# Compute SHA256(revocation_basepoint || per_commitment_point)
sha_output = sha256(bytes.fromhex(bob_revocation_basepoint_pubkey) + bytes.fromhex(alice_per_commitment_point))
sha_int = int.from_bytes(sha_output, 'big') % n
# now multiply it by revocation_basepoint
part1 = decompress_point(bob_revocation_basepoint_pubkey) * sha_int
# Compute SHA256(per_commitment_point || revocation_basepoint)
sha_output = sha256(bytes.fromhex(alice_per_commitment_point) + bytes.fromhex(bob_revocation_basepoint_pubkey))
sha_int = int.from_bytes(sha_output, 'big') % n
# multiply it by per_commitment_point
part2 = decompress_point(bob_per_commitment_point) * sha_int
# sum the results
bob_revocation_pubkey_point = part1 + part2
bob_revocation_pubkey = compress_pubkey(bob_revocation_pubkey_point)
print(f"Bob Revocation Pubkey: {bob_revocation_pubkey }")



Alice Delayed Pubkey: 03eb397e72815b5221b63d2564593465b45c9f7fa6da6944e1371974557d2682fa
Bob Delayed Pubkey: 02422c467da01480bef0191075e8823724ebb84b1a8fc502ca1e63d881ae723a4e
Alice HTLC Pubkey: 03f020b85b2f600cdba67ecb33ed2ed09bf1c0a0e67e839a9ca70766049a5f3298
Bob HTLC Pubkey: 03f0ce4fe88d9e1d79c449ced61ee6cfc9280b3e424a48f952667b7fd73a299aa1
Alice Revocation Pubkey: 02983e96a084377dc9d3c74cf49ae7e42f0347535d195d569f8b65c20341a8a2ad
Bob Revocation Pubkey: 02337f3fcfa5c9aef367ca7e3b275f7d7b539c821d73adc3bd75b6bf872321aa58


In [5]:
# OUTPUTS FOR ALICE COMMITMENT TRANSACTION
# 0x05 outputs
output_count = bytes.fromhex("05")

# ANCHOR AMOUNT OUTPUT
anchor_output_value = 330

# ANCHOR P2WSH
to_alice_anchor_redeemScript = bytes.fromhex(
    "21"
    + alice_funding_pubkey
    + "ac"  # OP_CHECKSIG
    + "73"  # OP_IFDUP
    + "64"  # OP_NOTIF
    + "60"  # OP_16
    + "B2"  # OP_CHECKSEQUENCEVERIFY
    + "68") # OP_ENDIF

to_alice_anchor_script_hash = sha256(to_alice_anchor_redeemScript)
to_alice_anchor_output_spk = bytes.fromhex("0020") + to_alice_anchor_script_hash
print(f"Alice Anchor Output SPK: {to_alice_anchor_output_spk.hex()}")

# ANCHOR P2WSH
to_bob_anchor_redeemScript = bytes.fromhex(
    "21"
    + bob_funding_pubkey
    + "ac"  # OP_CHECKSIG
    + "73"  # OP_IFDUP
    + "64"  # OP_NOTIF
    + "60"  # OP_16
    + "b2"  # OP_CHECKSEQUENCEVERIFY
    + "68") # OP_ENDIF

to_bob_anchor_script_hash = sha256(to_bob_anchor_redeemScript)
to_bob_anchor_output_spk = bytes.fromhex("0020") + to_bob_anchor_script_hash
print(f"Bob Anchor Output SPK: {to_bob_anchor_output_spk.hex()}")

# OFFERED HTLC OUTPUT
# To remote node with revocation key
# OP_DUP OP_HASH160 <RIPEMD160(SHA256(revocationpubkey))> OP_EQUAL
# OP_IF
#    OP_CHECKSIG
# OP_ELSE
#    <remote_htlcpubkey> OP_SWAP OP_SIZE 32 OP_EQUAL
#    OP_NOTIF
#        # To local node via HTLC-timeout transaction (timelocked).
#        OP_DROP 2 OP_SWAP <local_htlcpubkey> 2 OP_CHECKMULTISIG
#    OP_ELSE
#        To remote node with preimage.
#        OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY
#        OP_CHECKSIG
#    OP_ENDIF
#    1 OP_CHECKSEQUENCEVERIFY OP_DROP
# OP_ENDIF

htlc_output_value = 500000

hash160_bob_revocation_pubkey = hash160(bytes.fromhex(bob_revocation_pubkey))
payment_hash = hash160(bytes.fromhex("01"))

alice_offered_htlc_redeemScript = bytes.fromhex(
    "76"    # OP_DUP
    + "a9"  # OP_HASH160
    + "14"  # OP_PUSHDATA
    + hash160_bob_revocation_pubkey.hex()
    + "87"  # OP_EQUAL
    + "63"  # OP_IF
    + "ac"  # OP_CHECKSIG
    + "67"  # OP_ELSE
    + "21"  # OP_PUSHDATA
    + bob_htlc_pubkey
    + "7c"  # OP_SWAP
    + "82"  # OP_SIZE
    + "01"  # OP_PUSHDATA
    + "20"  # 32
    + "87"  # OP_EQUAL
    + "64"  # OP_NOTIF
    + "75"  # OP_DROP
    + "52"  # OP_2
    + "7c"  # OP_SWAP
    + "21"  # OP_PUSHDATA
    + alice_htlc_pubkey
    + "52"  # OP_2
    + "ae"  # OP_CHECKMULTISIG
    + "67"  # OP_ELSE
    + "a9"  # OP_HASH160
    + "14"  # OP_PUSHDATA
    + payment_hash.hex()
    + "88"  # OP_EQUALVERIFY
    + "ac"  # OP_CHECKSIG
    + "68"  # OP_ENDIF
    + "51"  # OP_1
    + "b2"  # OP_CHECKSEQUENCEVERIFY
    + "75"  # OP_DROP
    + "68") # OP_ENDIF

alice_offered_htlc_script_hash = sha256(alice_offered_htlc_redeemScript)
alice_offered_htlc_output_spk = bytes.fromhex("0020") + alice_offered_htlc_script_hash
print(f"Alice Offered htlc Output SPK: {alice_offered_htlc_output_spk.hex()}")    

# TO REMOTE OUTPUT
# <remotepubkey> OP_CHECKSIGVERIFY 1 OP_CHECKSEQUENCEVERIFY

bob_full_output_value = 1000000

to_bob_remote_redeemScript = bytes.fromhex(
    "21"   # OP_PUSHDATA
    + bob_payment_basepoint_pubkey
    + "ad"   # OP_CHECKSIGVERIFY
    + "51"   # OP_1
    + "b2")  # OP_CHECKSEQUENCEVERIFY

to_bob_remote_script_hash = sha256(to_bob_remote_redeemScript)
to_bob_remote_output_spk = bytes.fromhex("0020") + to_bob_remote_script_hash
print(f"Bob Remote Output SPK: {to_bob_remote_output_spk.hex()}") 
                             
# TO_LOCAL OUTPUT
# This output sends funds back to the owner of this commitment transaction and thus must be timelocked
# using OP_CHECKSEQUENCEVERIFY. It can be claimed, without delay, by the other party if they know the
# revocation private key. The output is a version-0 P2WSH, with the witness script below:
# OP_IF
    # Penalty transaction
#    <revocationpubkey>
# OP_ELSE
#    `to_self_delay`
#    OP_CHECKSEQUENCEVERIFY
#    OP_DROP
#    <local_delayedpubkey>
# OP_ENDIF
# OP_CHECKSIG

alice_full_output_value = 3500000

to_alice_delayed_redeemScript = bytes.fromhex(
    "63"   # OP_IF
    + "21"
    + bob_revocation_pubkey
    + "67"   # OP_ELSE
    + "02"   
    + "9000" # to_self_delay of 144 blocks
    + "b2"   # OP_CHECKSEQUENCEVERIFY
    + "75"   # OP_DROP
    + "21"
    + alice_delayed_pubkey
    + "68"  # OP_ENDIF
    + "ac") # OP_CHECKSIG

to_alice_delayed_script_hash = sha256(to_alice_delayed_redeemScript)
to_alice_delayed_output_spk = bytes.fromhex("0020") + to_alice_delayed_script_hash
print(f"Alice Delayed Output SPK: {to_alice_delayed_output_spk.hex()}")


Alice Anchor Output SPK: 002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04
Bob Anchor Output SPK: 002056680a199b598b249d54e3b24fbf5a629228af306e3f587e8ab1b65233a581c4
Alice Offered htlc Output SPK: 0020ea9713c7f38440f6ace2e3ad1b4982821623577e3d69908b76c0ce40f140602b
Bob Remote Output SPK: 0020439fd0fb44bb93e02a87b92851c9e377612166d8ce4af46eb9f82899a8e8323d
Alice Delayed Output SPK: 0020ba1d69954241e722f06844375d5b4dcf7f3abfcce550e708bf0910383060d31a


In [6]:
# OUTPUTS FOR BOB COMMITMENT TRANSACTION
# The number of outputs are the same
# The anchors outputs are the same

# RECEIVED HTLC OUTPUT
# To remote node with revocation key
# OP_DUP OP_HASH160 <RIPEMD160(SHA256(revocationpubkey))> OP_EQUAL
# OP_IF
#    OP_CHECKSIG
# OP_ELSE
#    <remote_htlcpubkey> OP_SWAP OP_SIZE 32 OP_EQUAL
#    OP_IF
#       To local node via HTLC-success transaction.
#       OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY
#       2 OP_SWAP <local_htlcpubkey> 2 OP_CHECKMULTISIG
#    OP_ELSE
#       To remote node after timeout.
#       OP_DROP <cltv_expiry> OP_CHECKLOCKTIMEVERIFY OP_DROP
#       OP_CHECKSIG
#    OP_ENDIF
#    1 OP_CHECKSEQUENCEVERIFY OP_DROP
# OP_ENDIF
hash160_alice_revocation_pubkey = hash160(bytes.fromhex(alice_revocation_pubkey))
payment_hash = hash160(bytes.fromhex("01"))

bob_received_htlc_redeemScript = bytes.fromhex(
    "76"    # OP_DUP
    + "a9"  # OP_HASH160
    + "14"  # OP_PUSHDATA
    + hash160_alice_revocation_pubkey.hex()
    + "87"  # OP_EQUAL
    + "63"  # OP_IF
    + "ac"  # OP_CHECKSIG
    + "67"  # OP_ELSE
    + "21"  # OP_PUSHDATA
    + alice_htlc_pubkey
    + "7c"  # OP_SWAP
    + "82"  # OP_SIZE
    + "01"  # OP_PUSHDATA
    + "20"  # 32
    + "87"  # OP_EQUAL
    + "63"  # OP_IF
    + "a9"  # OP_HASH160
    + payment_hash.hex()
    + "88"  # OP_EQUALVERIFY
    + "52"  # OP_2
    + "7c"  # OP_SWAP
    + "21"  # OP_PUSHDATA
    + bob_htlc_pubkey
    + "52"  # OP_2
    + "ae"  # OP_CHECKMULTISIG
    + "67"  # OP_ELSE
    + "75"  # OP_DROP
    + "02"  # OP_PUSHDATA 
    + "f401"# htlc expiry
    + "b1"  # OP_CHECKLOCKTIMEVERIFY
    + "75"  # OP_DROP
    + "ac"  # OP_CHECKSIG
    + "68"  # OP_ENDIF
    + "51"  # OP_1
    + "b2"  # OP_CHECKSEQUENCEVERIFY
    + "75"  # OP_DROP
    + "68") # OP_ENDIF

bob_received_htlc_script_hash = sha256(bob_received_htlc_redeemScript)
bob_received_htlc_output_spk = bytes.fromhex("0020") + bob_received_htlc_script_hash
print(f"Bob Received htlc Output SPK: {bob_received_htlc_output_spk.hex()}")    

# TO_LOCAL OUTPUT
# This output sends funds back to the owner of this commitment transaction and thus must be timelocked
# using OP_CHECKSEQUENCEVERIFY. It can be claimed, without delay, by the other party if they know the
# revocation private key. The output is a version-0 P2WSH, with the witness script below:
# OP_IF
    # Penalty transaction
#    <revocationpubkey>
# OP_ELSE
#    `to_self_delay`
#    OP_CHECKSEQUENCEVERIFY
#    OP_DROP
#    <local_delayedpubkey>
# OP_ENDIF
# OP_CHECKSIG

to_bob_delayed_redeemScript = bytes.fromhex(
    "63"   # OP_IF
    + "21"
    + alice_revocation_pubkey
    + "67"   # OP_ELSE
    + "02"   
    + "9000" # to_self_delay of 144 blocks
    + "b2"   # OP_CHECKSEQUENCEVERIFY
    + "75"   # OP_DROP
    + "21"
    + bob_delayed_pubkey
    + "68"  # OP_ENDIF
    + "ac") # OP_CHECKSIG

to_bob_delayed_script_hash = sha256(to_bob_delayed_redeemScript)
to_bob_delayed_output_spk = bytes.fromhex("0020") + to_bob_delayed_script_hash
print(f"Bob Delayed Output SPK: {to_bob_delayed_output_spk.hex()}")

# TO REMOTE OUTPUT
# <remotepubkey> OP_CHECKSIGVERIFY 1 OP_CHECKSEQUENCEVERIFY
to_alice_remote_redeemScript = bytes.fromhex(
    "21"   # OP_PUSHDATA
    + alice_payment_basepoint_pubkey
    + "ad"   # OP_CHECKSIGVERIFY
    + "51"   # OP_1
    + "b2")  # OP_CHECKSEQUENCEVERIFY

to_alice_remote_script_hash = sha256(to_alice_remote_redeemScript)
to_alice_remote_output_spk = bytes.fromhex("0020") + to_alice_remote_script_hash
print(f"Alice Remote Output SPK: {to_alice_remote_output_spk.hex()}") 

Bob Received htlc Output SPK: 0020bb9fcdc5ed7ac56775f98430524160bfdaad50ee7add4650618f77b0a3113ed9
Bob Delayed Output SPK: 002022d5ac3f86d25be331276291c7cd8aeb71e005fc78ccc40e81779cc9ed17477a
Alice Remote Output SPK: 0020475d024bb95e9a981632863d3b1a4b9308aa88e189196272735a3e3d47caf936


### Fee Calculation

The fee calculation for commitment transactions  is based on the current `feerate_per_kw` sent by Alice into the `open_channel` message and the **expected** weight of the transaction.

The Basis of Lightning Technology [BOLT](https://github.com/lightning/bolts/blob/master/03-transactions.md#fees) defines the following for the expected weights:
* Commitment weight (no option_anchors):   724 + 172 * num-untrimmed-htlc-outputs
* Commitment weight (option_anchors):     1124 + 172 * num-untrimmed-htlc-outputs
* HTLC-timeout weight (no option_anchors): 663
* HTLC-timeout weight (option_anchors): 666
* HTLC-success weight (no option_anchors): 703
* HTLC-success weight (option_anchors): 706

### Fee Payment

Base commitment transaction fees and amounts for to_local_anchor and to_remote_anchor outputs are extracted from the funder's amount; Restrictions to the commitment tx output for the funder in relation to the channel reserve apply as described in [BOLT #2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md).

In [7]:
# We are going to use the same fee for both transactions
commitment_weight = 1124 + 172 * 1

# data from open_channel message
feerate_per_kw = 15000
funding_satoshis = 5000000

commitment_fee_rate = int(commitment_weight * feerate_per_kw / 1000)

alice_output_value = funding_satoshis - anchor_output_value * 2 - bob_full_output_value - htlc_output_value - commitment_fee_rate

to_alice_output_value = alice_output_value.to_bytes(8, byteorder="little", signed=True)
to_bob_output_value = bob_full_output_value.to_bytes(8, byteorder="little", signed=True)
alice_anchor_output_value = anchor_output_value.to_bytes(8, byteorder="little", signed=True)
bob_anchor_output_value = anchor_output_value.to_bytes(8, byteorder="little", signed=True)
alice_offered_htlc_output_value = htlc_output_value.to_bytes(8, byteorder="little", signed=True)


outputs_alice_commitment = (
    alice_anchor_output_value
    + varint_len(to_alice_anchor_output_spk)
    + to_alice_anchor_output_spk
    + bob_anchor_output_value
    + varint_len(to_bob_anchor_output_spk)
    + to_bob_anchor_output_spk
    + alice_offered_htlc_output_value
    + varint_len(alice_offered_htlc_output_spk)
    + alice_offered_htlc_output_spk
    + to_bob_output_value
    + varint_len(to_bob_remote_output_spk)
    + to_bob_remote_output_spk
    + to_alice_output_value
    + varint_len(to_alice_delayed_output_spk)
    + to_alice_delayed_output_spk
)

bob_received_htlc_output_value = htlc_output_value.to_bytes(8, byteorder="little", signed=True)

outputs_bob_commitment = (
    alice_anchor_output_value
    + varint_len(to_alice_anchor_output_spk)
    + to_alice_anchor_output_spk
    + bob_anchor_output_value
    + varint_len(to_bob_anchor_output_spk)
    + to_bob_anchor_output_spk
    + bob_received_htlc_output_value
    + varint_len(bob_received_htlc_output_spk)
    + bob_received_htlc_output_spk
    + to_bob_output_value
    + varint_len(to_bob_delayed_output_spk)
    + to_bob_delayed_output_spk
    + to_alice_output_value
    + varint_len(to_alice_remote_output_spk)
    + to_alice_remote_output_spk
)

# Locktime: upper 8 bits are 0x20, lower 24 bits are the lower 24 bits of the obscured commitment number
# Extract the lower 24 bits of the obscured commitment number
lower_24_bits = commitment_number_obscured & 0xFFFFFF
# Combine the upper 8 bits (0x20) with the lower 24 bits (lower 24 of obscured number)
locktime = (0x20 << 24) | lower_24_bits
# Convert to bytes (1 byte, big-endian)
locktime = locktime.to_bytes(4, byteorder='big')
locktime = locktime[::-1]

unsigned_alice_commitment_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs_alice_commitment
    + locktime
)

unsigned_bob_commitment_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs_bob_commitment
    + locktime
)
hash = hash256(unsigned_alice_commitment_tx)
print("TXID :", hash.hex())
print("Unsigned Alice Commitment Transaction: ", unsigned_alice_commitment_tx.hex())
print("Unsigned Bob Commitment Transaction: ", unsigned_bob_commitment_tx.hex())

TXID : 0ed43d8b4de80d18e6c6855c967a6f5409baa0cc4d5a3a75a3d71d8dee9f4192
Unsigned Alice Commitment Transaction:  020000000158daedb84cb5a96c3d7899fed40160e9fc980168b82c42670fdb9de6820b18ca0100000000fd33b480054a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f044a0100000000000022002056680a199b598b249d54e3b24fbf5a629228af306e3f587e8ab1b65233a581c420a1070000000000220020ea9713c7f38440f6ace2e3ad1b4982821623577e3d69908b76c0ce40f140602b40420f0000000000220020439fd0fb44bb93e02a87b92851c9e377612166d8ce4af46eb9f82899a8e8323d5c19350000000000220020ba1d69954241e722f06844375d5b4dcf7f3abfcce550e708bf0910383060d31a6ca64320
Unsigned Bob Commitment Transaction:  020000000158daedb84cb5a96c3d7899fed40160e9fc980168b82c42670fdb9de6820b18ca0100000000fd33b480054a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f044a0100000000000022002056680a199b598b249d54e3b24fbf5a629228af306e3f587e8ab1b65233a581c420a1070000000000220020bb9fcdc5ed7ac56775f9843052416

We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [8]:
decoded = node.decoderawtransaction(unsigned_alice_commitment_tx.hex())
print("Alice Unsigned Commitment transaction")
print(json.dumps(decoded, indent=2, default=str))
decoded = node.decoderawtransaction(unsigned_bob_commitment_tx.hex())
print("Bob Unsigned Commitment transaction")
print(json.dumps(decoded, indent=2, default=str))

Alice Unsigned Commitment transaction
{
  "txid": "92419fee8d1dd7a3753a5a4dcca0ba09546f7a965c85c6e6180de84d8b3dd40e",
  "hash": "92419fee8d1dd7a3753a5a4dcca0ba09546f7a965c85c6e6180de84d8b3dd40e",
  "version": 2,
  "size": 266,
  "vsize": 266,
  "weight": 1064,
  "locktime": 541304428,
  "vin": [
    {
      "txid": "ca180b82e69ddb0f67422cb8680198fce96001d4fe99783d6ca9b54cb8edda58",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 2159293437
    }
  ],
  "vout": [
    {
      "value": "0.00000330",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 34391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04",
        "desc": "addr(bcrt1qxsu3c7d60r2egf324y2ppayawgqw7408252yzev5z24vg7658uzqwq2zs5)#62txjpcp",
        "hex": "002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f04",
        "address": "bcrt1qxsu3c7d60r2egf324y2ppayawgqw7408252yzev5z24vg7658uzqwq2zs5",
        "type": "witness_v0_scripthash"
      }


In [9]:
# value of the funding channel output transaction (Output2)
value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(channel_funding_txid + channel_funding_txindex)
hashSequence = hash256(sequence)
hashAliceOutputs = hash256(outputs_alice_commitment)
hashBobOutputs = hash256(outputs_bob_commitment)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

alice_tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + channel_funding_txid
    + channel_funding_txindex
    + varint_len(redeemScript) # size of scriptcode
    + redeemScript             # scriptcode is the redeemscript of the Output2 (channel funding)
    + value
    + sequence
    + hashAliceOutputs
    + locktime
    + sighash_type
)

bob_tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + channel_funding_txid
    + channel_funding_txindex
    + varint_len(redeemScript) # size of scriptcode
    + redeemScript             # scriptcode is the redeemscript of the Output2 (channel funding)
    + value
    + sequence
    + hashBobOutputs
    + locktime
    + sighash_type
)
print("alice_commitment_tx_digest_preimage: ",alice_tx_digest_preimage.hex())
print("bob_commitment_tx_digest_preimage: ",bob_tx_digest_preimage.hex())

alice_commitment_tx_digest_preimage:  02000000fe91c64b9d426387907c8c158a5f82c55f739a67be7e0a9492760fa2822f4285c6067e2ccbba74d4f24cc174be97c1ea7514cf7a20d616fbc7587a48964b2c9558daedb84cb5a96c3d7899fed40160e9fc980168b82c42670fdb9de6820b18ca0100000047522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae404b4c0000000000fd33b4806c57f53984d4f1f87befc46fc3ea04f88e73aa9fa648504b761825153e37d2486ca6432001000000
bob_commitment_tx_digest_preimage:  02000000fe91c64b9d426387907c8c158a5f82c55f739a67be7e0a9492760fa2822f4285c6067e2ccbba74d4f24cc174be97c1ea7514cf7a20d616fbc7587a48964b2c9558daedb84cb5a96c3d7899fed40160e9fc980168b82c42670fdb9de6820b18ca0100000047522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae404b4c0000000000fd33b48006d578cddd13b53a4f305650c917bbdf2764c75da81394249815db709fb156e56ca6432001000000


Segwit transactions have a signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)

In [10]:
# SIGN ALICE COMMITMENT TRANSACTION
# Sign the sigHash with the input privkey1
# Create sigHash to be signed
alice_sighash = hash256(alice_tx_digest_preimage)
alice_signing_key1 = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
alice_signature1 = alice_signing_key1.sign_digest(alice_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
alice_signature1 = alice_signature1 + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
alice_signing_key2 = ecdsa.SigningKey.from_string(bytes.fromhex(bob_funding_privkey), curve=ecdsa.SECP256k1) 
alice_signature2 = alice_signing_key2.sign_digest(alice_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
alice_signature2 = alice_signature2 + bytes.fromhex("01")

# Combine the signatures into the final scriptSig
alice_witness = (
    # indicate the number of stack items
    bytes.fromhex("04")
    + bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + varint_len(alice_signature1)
    + alice_signature1
    + varint_len(alice_signature2)
    + alice_signature2
    + varint_len(redeemScript)
    + redeemScript
)

# the final signed transaction
alice_signed_commitment_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs_alice_commitment
    + alice_witness
    + locktime
)

# SIGN BOB COMMITMENT TRANSACTION
# Sign the sigHash with the input privkey1
# Create sigHash to be signed
bob_sighash = hash256(bob_tx_digest_preimage)
bob_signing_key1 = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
bob_signature1 = bob_signing_key1.sign_digest(bob_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
bob_signature1 = bob_signature1 + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
bob_signing_key2 = ecdsa.SigningKey.from_string(bytes.fromhex(bob_funding_privkey), curve=ecdsa.SECP256k1) 
bob_signature2 = bob_signing_key2.sign_digest(bob_sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
bob_signature2 = bob_signature2 + bytes.fromhex("01")

# Combine the signatures into the final scriptSig
bob_witness = (
    # indicate the number of stack items
    bytes.fromhex("04")
    + bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + varint_len(bob_signature1)
    + bob_signature1
    + varint_len(bob_signature2)
    + bob_signature2
    + varint_len(redeemScript)
    + redeemScript
)

# the final signed transaction
bob_signed_commitment_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs_bob_commitment
    + bob_witness
    + locktime
)

print("Alice Signed Commitment Transaction: ",alice_signed_commitment_tx.hex())
print("Bob Signed Commitment Transaction: ",bob_signed_commitment_tx.hex())

Alice Signed Commitment Transaction:  0200000000010158daedb84cb5a96c3d7899fed40160e9fc980168b82c42670fdb9de6820b18ca0100000000fd33b480054a0100000000000022002034391c79ba78d594262aa91410f49d7200ef55e7551441659412aac47b543f044a0100000000000022002056680a199b598b249d54e3b24fbf5a629228af306e3f587e8ab1b65233a581c420a1070000000000220020ea9713c7f38440f6ace2e3ad1b4982821623577e3d69908b76c0ce40f140602b40420f0000000000220020439fd0fb44bb93e02a87b92851c9e377612166d8ce4af46eb9f82899a8e8323d5c19350000000000220020ba1d69954241e722f06844375d5b4dcf7f3abfcce550e708bf0910383060d31a04004730440220091bf62dcb30b82f9f91a0116fc59a3177312942d6d1479c3f31963c29c9782202205fcac6e2dda9e0ec7a6f49e3c8acdb3bfad9ccbf95e1149a9f12a74118653cee01483045022100eb7bf5cd11a82caf2c4aa9f263706fd51521da928001fffdbf19ea7d2153c5e202205c0e7de8f82b2a6904dc1183c5ec772203b8871f2a60070a62df2eccf1e504010147522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c3

In [11]:
print(node.testmempoolaccept(rawtxs=[alice_signed_commitment_tx.hex()]))

print(node.testmempoolaccept(rawtxs=[bob_signed_commitment_tx.hex()]))

[{'txid': '92419fee8d1dd7a3753a5a4dcca0ba09546f7a965c85c6e6180de84d8b3dd40e', 'wtxid': '98f9f53db398c5b01c70a017a5dfa7afd372a52126ea3e45fe4a1fa91ba90c40', 'allowed': True, 'vsize': 322, 'fees': {'base': Decimal('0.00019440'), 'effective-feerate': Decimal('0.00060372'), 'effective-includes': ['98f9f53db398c5b01c70a017a5dfa7afd372a52126ea3e45fe4a1fa91ba90c40']}}]
[{'txid': '2f9781a070a63898c1940391187af4821b33c27c2f1cf07d79fcc2a49b9faade', 'wtxid': '4fa69eedef3d45324157ce9677f5387becd9d45405542db1300750d375299a46', 'allowed': True, 'vsize': 322, 'fees': {'base': Decimal('0.00019440'), 'effective-feerate': Decimal('0.00060372'), 'effective-includes': ['4fa69eedef3d45324157ce9677f5387becd9d45405542db1300750d375299a46']}}]
